# Fighter Calibration Model

## Configure your Fighters here: 

In [ ]:
# Define Fighter 1
fighter1_name = "Fighter #1"
fighter1_attack_range = [1,60]
fighter1_speed_range = [1,23]

fighter1_defense_range = [1,25]
fighter1_stamina_range = [1,40]


# Define Fighter 2
fighter2_name = "Fighter #2"
fighter2_attack_range = [1,75]
fighter2_speed_range = [1,27]

fighter2_defense_range = [1,57]
fighter2_stamina_range = [1,23]


number_of_fights = 1000
initial_health = 150

## Execute Fight(s)

In [ ]:
from Objects.Fighter import Fighter
from Objects.Fight import Fight

In [ ]:
for number in range(number_of_fights):
    f1 = Fighter(fighter1_name, fighter1_attack_range, fighter1_defense_range, fighter1_stamina_range, fighter1_speed_range, initial_health)
    f2 = Fighter(fighter2_name, fighter2_attack_range, fighter2_defense_range, fighter2_stamina_range, fighter2_speed_range, initial_health)
    
    f = Fight(number+1, f1, f2)

## Process Data

### Pick up and display sample of statistics file

In [ ]:
import pandas as pd

df = pd.read_csv('statistics.csv')
df.head(5)

### Summarize data and group

In [ ]:
# distribution of rounds
df_fightround = df[['fight_number','fight_round']].groupby('fight_number').max().reset_index()

# balance per round
f1_winner_balance_round = df[df.fighter1_win == True].shape[0]/len(df)
f2_winner_balance_round = df[df.fighter2_win == True].shape[0]/len(df)

# balance per fight
df_group = df.groupby(['fight_number'], sort=False)[['fight_round', 'fighter1_health_out', 'fighter2_health_out']].agg({'fighter1_health_out': ['min'], 'fighter2_health_out': ['min']})
df_group['winner'] = 2
df_group.loc[df_group['fighter1_health_out']['min'] > df_group['fighter2_health_out']['min'], 'winner'] = 1

f1_winner_balance_fight = df_group[df_group.winner == 1].shape[0]/len(df_group)
f2_winner_balance_fight = df_group[df_group.winner == 2].shape[0]/len(df_group)

# fighter profile chart
profile1 = []

## Generate Chart

In [ ]:
# import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline
import numpy as np
from datetime import datetime
from math import pi

fig = plt.figure(figsize=(15,20))

# Configure a 4x2 grid
# +-------+
# |   1   |
# +---+---+
# | 2 | 3 |
# +---+---+
# | 4 | 5 |
# +---+---+
# |   6   |
# +-------+

gs = GridSpec(4, 2, figure=fig)

ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, -1])
ax4 = fig.add_subplot(gs[2, 0])
ax5 = fig.add_subplot(gs[2, -1])
ax6 = fig.add_subplot(gs[-1, :])

# Axes 1
# Remove grid
ax1.axis('off')
# Set title
title = fig.text(0.5, 0.965, 'Fighter Calibration Model\n|Simulation of %i fights|' % number_of_fights,
                 horizontalalignment='center', color='black', weight='bold', fontsize='36')

# Display Fighter specs
#ax1.text(0.1, 0.5, 'Begin text', horizontalalignment='center', verticalalignment='center', transform=ax1.transAxes)
f1_text = ax1.text(0.5, 0.7, f1.__str__(), horizontalalignment='center', fontsize='22')
vs_text = ax1.text(0.5, 0.6, 'vs.', horizontalalignment='center', fontsize='22')
f2_text = ax1.text(0.5, 0.5, f2.__str__(), horizontalalignment='center', fontsize='22')
dt_text = ax1.text(1, 0.3, 'run on %s' % (datetime.now().strftime("%B %d, %Y at %H:%M")), horizontalalignment='right', fontsize='12')

# Axes 2 - distribution 
sns.set(style="darkgrid", font_scale=1.2)

chart = sns.histplot(data=df_max,
                     x="fight_round",
                     ax=ax2, 
                     discrete=True)

chart_ticks = chart.set(xticks=[1,2,3,4,5,6,7,8,9,10],
                        xlabel="Rounds (Distance)",
                        ylabel="Frequency",
                        title="Distance Distribution",
                        xlim=(0,11))

# Axes 3

v = pd.DataFrame(df_max['fight_round'].value_counts()).sort_index()
ax3.axis('off')
table = ax3.table(cellText = v.values, rowLabels = v.index, bbox=[0,0,1,1], colLabels = ["Frequency"])

# Axes 4
round_pie = ax4.pie([f1_winner_balance_round, f2_winner_balance_round],
                    labels = ['Fighter 1', 'Fighter 2'], 
                    colors = ['#7e8cbe','#ff9999'], autopct='%.2f%%')

round_title = ax4.set_title('Rounds won')

# Axes 5
fight_pie = ax5.pie([f1_winner_balance_fight, f2_winner_balance_fight],
                    labels = ['Fighter 1', 'Fighter 2'], 
                    colors = ['#7e8cbe','#ff9999'], autopct='%.2f%%')

fight_title = ax5.set_title('Fights won')

# Axes 6
profile_title = ax6.set_title('Fighter Profile')

categories=['attack', 'speed', 'defense', 'stamina']
N = len(categories)

df_profile = pd.DataFrame({
    'name': [f1.name, f2.name],
    'attack':  [f1.attack.max, f2.attack.max],
    'speed':  [f1.speed.max, f2.speed.max],
    'defense': [f1.defense.max, f2.defense.max],
    'stamina':  [f1.stamina.max, f2.stamina.max]})

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

ax6 = plt.subplot(427, polar=True)

# position first axis at the top
ax6.set_theta_offset(pi / 2)
ax6.set_theta_direction(-1)

# Draw one axe per variable + add labels labels yet
plt.xticks(angles[:-1], categories, color='grey', size=20)

ax6.set_rlabel_position(0)
plt.yticks(np.arange(10,110,10), np.arange(10,110,10), color="grey", size=7)
plt.ylim(0,100)

# Fighter 1
values=df_profile.loc[0].drop('name').values.flatten().tolist()
values += values[:1]
f1_radar = ax6.plot(angles, values, linewidth=3, linestyle='solid', label=f1.name)
ax6.fill(angles, values, 'b', alpha=0.1)

# Fighter 2
values=df_profile.loc[1].drop('name').values.flatten().tolist()
values += values[:1]
f2_radar = ax6.plot(angles, values, linewidth=3, linestyle='solid', label=f2.name)
ax6.fill(angles, values, 'r', alpha=0.1)

plt.show()

## Move Statistics to Archive

In [ ]:
import os

counter = 0

# find the latest file number in archives, increase by 1
# if no file is found yet, start counter at 1
filelist = os.listdir("./Archives")
if  filelist == []:
    counter = 1
else:
    filelist.sort()
    counter = int(os.path.basename(filelist[-1])[0]) + 1

# save chart and table as PDF
fig.savefig("Archives/%i.pdf" % counter)

# move statistics file to archive
os.rename("statistics.csv", "Archives/%i.csv" % counter)